In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Text Analysis\n",
    "Simple text analysis example"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import libraries\n",
    "import pandas as pd\n",
    "import re\n",
    "import nltk\n",
    "from nltk.tokenize import word_tokenize\n",
    "from nltk.corpus import stopwords\n",
    "from nltk.stem import PorterStemmer\n",
    "from collections import Counter\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "# Download NLTK resources\n",
    "nltk.download('punkt')\n",
    "nltk.download('stopwords')\n",
    "nltk.download('wordnet')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Data contoh\n",
    "texts = [\n",
    "    \"I really love this product! It's amazing and works great.\",\n",
    "    \"The service was terrible, I hate it!\",\n",
    "    \"Good product but expensive price.\",\n",
    "    \"Customer service is excellent, very helpful!\"\n",
    "]\n",
    "\n",
    "# Buat DataFrame\n",
    "df = pd.DataFrame(texts, columns=['text'])\n",
    "print(\"Data awal:\")\n",
    "print(df)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Fungsi pembersihan\n",
    "def clean_text(text):\n",
    "    text = text.lower()\n",
    "    text = re.sub(r'[^a-zA-Z\\s]', '', text)\n",
    "    text = re.sub(r'\\s+', ' ', text).strip()\n",
    "    return text\n",
    "\n",
    "df['cleaned_text'] = df['text'].apply(clean_text)\n",
    "print(\"\\nHasil pembersihan teks:\")\n",
    "print(df[['text', 'cleaned_text']])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Tokenization\n",
    "df['tokens'] = df['cleaned_text'].apply(word_tokenize)\n",
    "print(\"\\nHasil tokenization:\")\n",
    "print(df[['cleaned_text', 'tokens']])"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}